In [2]:
from requests_futures.sessions import FuturesSession
from ediblepickle import checkpoint
import hashlib
session = FuturesSession(max_workers=5)

In [3]:
year = '2017'
game_type = '02'
game = '0090'
game_ID = f'{year}{game_type}{game}'
url = f'https://statsapi.web.nhl.com/api/v1/game/{game_ID}/feed/live'

In [4]:
def get_games_futures(year,max_games = 1200):
    '''
        in: string year (4-digit required)
        out: list of futures, one for each game that season, querying play-by-play data
    '''
    session = FuturesSession(max_workers=10)
    game_numbers = range(1,max_games)
    game_types = ['02',]
    futures = []
    for game_type in game_types:
        for game_num in game_numbers:
            game_number = str(game_num).zfill(4)
            game_ID = f'{year}{game_type}{game_number}'
            url = f'https://statsapi.web.nhl.com/api/v1/game/{game_ID}/feed/live'
            futures.append(session.get(url))
    return futures

In [5]:
def get_plays(future):
    '''
        in: future, a request to a nhl game live-feed 
        out: a list of plays from the game, where each play is a dictionary
            or None if no game
    '''
    d = future.result().json()
    if 'liveData' in d.keys():
        return [play for play in d['liveData']['plays']['allPlays']]
    else:
        return None
    

In [12]:
@checkpoint(key = lambda args, kwargs: (str(args[0])+'.p').encode(),
                work_dir='./cache'.encode(), 
                refresh=False)
def get_games_plays(year, futures=None):
    '''
        in: list of futures requesting nhl game live-feeds
        out: list of lists (one for each game) where inner list contains plays
    '''
    if not futures:
        futures = get_games_futures(year)
        print(f'{year} games')
    games_plays = []
    for i, future in enumerate(futures):
        print(f'\tgame {i}')
        plays = get_plays(future)
        if plays:
            games_plays.append(plays)
    return games_plays

In [155]:
def strip_event(play):
    '''
        in: full json for a play from nhl api
        out: bare bones dictionary with some features
    '''
    def getcoords(play_coordinates, sbin = 10):
        if not ('x' in play_coordinates.keys() 
                or 'y' in play_coordinates.keys()):
            return None
        x = None
        y = None
        if 'x' in play_coordinates.keys():
            x = int(play_coordinates['x']/sbin)*sbin
        if 'y' in play_coordinates.keys():
            y = int(play_coordinates['y']/sbin)*sbin
        if x or y:
            return (x, y)
        else:
            return None
    event = {}
    event['Type'] = ''.join(play['result']['event'].split())
    event['Coords'] = getcoords(play['coordinates'])
    event['periodTime'] = (int(play['about']['periodTime'][0:2])*60+
                           int(play['about']['periodTime'][3:]))
    return event

In [132]:
import numpy as np
def make_delta_time(game_events):
    '''
    in: a game's worth of plays
    out: modified list with feature expansion for time since last play
    '''
    def bin_DT(deltaTime):
        if deltaTime == 0:
            return -1
        else:
            return int(np.log2(deltaTime))
    for i, play in enumerate(game_events):
        if play['periodTime'] == 0:
            play['DeltaTime'] = -1
        else:
            play['DeltaTime'] = bin_DT(play['periodTime']-
                                       game_events[i-1]['periodTime'])

In [69]:
def strip_game_events(plays):
    '''
    in game, list of plays in nhl api json format
    out game, list of events in stripped dictionary format
    '''
    return [strip_event(play) for play in plays]

In [84]:
def process_games_plays(games):
    '''
    in: list of lists of plays
    out: list lists of stripped plays for each game
    '''
    games_stripped = [strip_game_events(game) for game in games]
    for game in games_stripped:
        make_delta_time(game)
    return games_stripped

In [100]:
def make_event_string(event):
    '''
    in: stripped event
    out: string
    '''
    coordstring = 'nocoords'
    if event['Coords']:
        coordstring = '{}_{}'.format(event['Coords'][0],
                                     event['Coords'][1])
        
    return '{}_{}_{}'.format(event['Type'],
                                coordstring,
                                event['DeltaTime'])

In [144]:
def games_to_corpus(games):
    '''
    in: a list of games in stripped/processed format
    out: a list of strings, one for each game, with each event 
        represented as a 'word' without whitespace
    '''
    corpus = []
    for game in games:
        events_string_list = []
        for event in game:
            events_string_list.append(make_event_string(event))
        corpus.append(' '.join(events_string_list))
    return corpus

In [127]:
#this will cache all the recent data on my server
#games_plays = []
for year in range(2018,2019):
    get_games_plays(year)

In [160]:
corpus = []
for year in range(2011,2019):
    games = get_games_plays(year)
    print(f'got {year} games')
    games_processed = process_games_plays(games)
    print('processed games')
    corpus.extend(games_to_corpus(games_processed))
    print('corpus extended')


got 2011 games
processed games
corpus extended
got 2012 games
processed games
corpus extended
got 2013 games
processed games
corpus extended
got 2014 games
processed games
corpus extended
got 2015 games
processed games
corpus extended
got 2016 games
processed games
corpus extended
got 2017 games
processed games
corpus extended
got 2018 games
processed games
corpus extended


In [165]:
with open('corpus.txt', 'w') as f:
    for game in corpus:
        f.write("%s\n" % game)

In [188]:
import logging
import sys
from collections import defaultdict
from gensim import corpora 
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [167]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [171]:
#format required for input to word2vec
game_string_lists = [game.split() for game in corpus]

In [173]:
game_string_lists[1:2][:]

[['GameScheduled_nocoords_-1',
  'PeriodReady_nocoords_-1',
  'PeriodStart_nocoords_-1',
  'Faceoff_nocoords_-1',
  'Stoppage_nocoords_4',
  'Faceoff_-20_-20_-1',
  'Giveaway_10_10_4',
  'MissedShot_-40_0_4',
  'Stoppage_nocoords_-1',
  'Faceoff_-60_-20_-1',
  'Hit_-70_-30_2',
  'Shot_30_-20_3',
  'Shot_60_20_3',
  'Hit_40_-30_4',
  'Hit_-90_0_4',
  'Hit_40_30_4',
  'MissedShot_-50_-20_3',
  'Giveaway_40_-30_3',
  'Shot_70_20_2',
  'Stoppage_nocoords_0',
  'Faceoff_60_20_-1',
  'Shot_-10_20_3',
  'Stoppage_nocoords_1',
  'Faceoff_-60_20_-1',
  'BlockedShot_80_0_3',
  'Stoppage_nocoords_3',
  'Faceoff_60_20_-1',
  'Stoppage_nocoords_-1',
  'Faceoff_60_20_3',
  'Stoppage_nocoords_-1',
  'Takeaway_80_30_2',
  'Shot_80_0_4',
  'Stoppage_nocoords_-1',
  'Faceoff_60_-20_-1',
  'Takeaway_-50_-30_5',
  'MissedShot_70_10_5',
  'Shot_40_0_3',
  'Shot_50_10_3',
  'Stoppage_nocoords_0',
  'Faceoff_60_20_-1',
  'Hit_70_-30_3',
  'Shot_-30_-30_4',
  'Takeaway_-90_10_3',
  'Stoppage_nocoords_4',
  'F

In [174]:
path = get_tmpfile("word2vec.model")

In [180]:
#train the word2vec encoding
model = Word2Vec(game_string_lists, size=100, window=5, min_count=1, workers=4)

In [196]:
model.save('mymodel.w2v')

In [217]:
model.wv['BlockedShot_60_0_2']

array([-6.52623028e-02,  2.62971252e-01,  3.30781221e-01,  7.79382065e-02,
       -3.76199335e-02,  5.98714590e-01,  6.38641790e-02,  3.17356974e-01,
        1.53100565e-01, -1.11597709e-01,  3.26891318e-02,  5.87340333e-02,
       -4.14673954e-01,  2.19534170e-02, -5.52453160e-01,  4.15992141e-02,
        2.54923642e-01, -5.99646032e-01,  1.09100066e-01,  9.53629520e-03,
       -2.21212264e-02, -1.39837533e-01, -2.70800799e-01,  5.32477796e-01,
        5.16480803e-01, -1.45630389e-01,  5.57392597e-01,  6.02832556e-01,
       -4.99377102e-01, -1.09522319e+00, -9.15114358e-02,  8.19948435e-01,
       -1.32371441e-01, -4.25279826e-01, -1.79876655e-01,  4.08810824e-01,
       -5.80217242e-01,  3.18313837e-01, -7.53280163e-01, -6.20947361e-01,
       -5.29556751e-01,  4.09418702e-01,  2.99303681e-01,  8.43272195e-04,
        6.85069084e-01,  1.30225718e-02,  1.20573330e+00,  1.17890239e-01,
       -3.54548216e-01, -4.25401717e-01, -1.10987902e-01, -6.03261180e-02,
       -8.04263800e-02,  